## Stable Diffusion Text-to-Image Generation on IPU

This notebook demonstrates how a stable diffusion inference pipeline can be run on Graphcore IPUs.

![Text to image on stable diffusion](sample_images/text_to_image.png)

### Requirements

* An enabled Poplar SDK environment (or Paperspace account with access to the PyTorch IPU runtime)
* Additional dependencies installable via pip (done below)
* Access to the pretrained Stable-Diffusion-v1-5 checkpoint (done below)

In [ ]:
%%capture
!pip install -r requirements.txt
!pip install "ipywidgets>=7,<8"

Values for the virtual IPU Pod size and the cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod4")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/stablediffusion_to-image"

To download the pretrained Stable-Diffusion-v1-5 checkpoint, we must first authenticate to the Hugging Face Hub. Begin by creating a read access token on the [Hugging Face website](https://huggingface.co/settings/tokens) (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your read token:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

If you have not done so already, you will need to accept the User License on the [model page](https://huggingface.co/runwayml/stable-diffusion-v1-5).

### Pipeline creation

We are now ready to import and run the pipeline.

In [ ]:
import torch

from ipu_models import get_default_ipu_configs, INFERENCE_ENGINES_TO_MODEL_NAMES, IPUStableDiffusionPipeline

In [ ]:
engine = "stable-diffusion-v1-5"  # maps to "runwayml/stable-diffusion-v1-5"
model_name = INFERENCE_ENGINES_TO_MODEL_NAMES[engine]
image_width = os.getenv("STABLE_DIFFUSION_TXT2IMG_DEFAULT_WIDTH", default=512)
image_height = os.getenv("STABLE_DIFFUSION_TXT2IMG_DEFAULT_HEIGHT", default=512)

In [ ]:
unet_ipu_config, text_encoder_ipu_config, vae_ipu_config, safety_checker_ipu_config = \
get_default_ipu_configs(
    engine=engine, width=image_width, height=image_height, pod_type=pod_type, 
    executable_cache_dir=executable_cache_dir 
)
pipe = IPUStableDiffusionPipeline.from_pretrained(
    model_name,
    revision="fp16", 
    torch_dtype=torch.float16,
    unet_ipu_config=unet_ipu_config,
    text_encoder_ipu_config=text_encoder_ipu_config,
    vae_ipu_config=vae_ipu_config,
    safety_checker_ipu_config=safety_checker_ipu_config
)
pipe.enable_attention_slicing()

We run a dummy generation step to trigger the one-time compilation process. This should take on the order of 15 minutes.

In [ ]:
pipe("apple", height=image_height, width=image_width, guidance_scale=7.5);

### Image generation

Below you will find some example prompts. We encourage you to try your own!

In [ ]:
prompt = "a shiba inu in a zen garden, acrylic painting"
pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5).images[0]

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5).images[0]

In [ ]:
prompt = "the living room of a cozy wooden house with a fireplace"
out = pipe(prompt, height=image_height, width=image_width, guidance_scale=7.5)
out.images[0]

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1,1)
fig.set_size_inches(9, 9)
ax.imshow(out.images[0])

ax.set_title(f"Prompt: {prompt}")
ax.axis("off")
fig.savefig("sample_images/text_to_image.png", dpi=150)

### Optional - Release IPUs in use

The IPython kernel has a lock on the IPUs used in running the model, preventing other users from using them. For example, if you wish to use other notebooks after working your way through this one, it may be necessary to manually run the below cell to release IPUs from use. This will happen by default if using the `Run All` option. More information on the topic can be found at [Managing IPU Resources](https://github.com/gradient-ai/Graphcore-HuggingFace/blob/main/useful-tips/managing_ipu_resources.ipynb).

In [ ]:
pipe.detach_from_device()